In [1]:
import torch
import copy, os
import logging
import numpy as np
import out_manager as om
from config import Config
import torch.nn.functional as F
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score, average_precision_score
from model.score_gnn import ScoreGNN

In [2]:
config = Config()
out_dir = om.get_out_dir(config)
log_path = os.path.join(out_dir, "scoregnn_log.txt")
om.setup_logging(log_path)
seed = config.seed
torch.manual_seed(seed)
np.random.seed(seed)
device = config.device

In [3]:
def train(model, data, optimizer, predictor):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)

    # #每次手动采样负样本，效果更好
    # neg_edge_index = negative_sampling(
    #     edge_index = data.edge_index,
    #     num_nodes = data.num_nodes,
    #     num_neg_sample = data.edge_index.size(1)
    # )

    # edge_label_index = torch.cat([
    #     data.edge_label_index, neg_edge_index
    #     ], dim=-1)
    # edge_label = torch.cat([
    #     data.edge_label, data.edge_label.new_zeros(neg_edge_index.size(1))
    #     ],dim = 0)

    edge_label_index = torch.cat([
        data.pos_edge_label_index, data.neg_edge_label_index
    ], dim=1)
    edge_label = torch.cat([
        data.pos_edge_label, data.neg_edge_label
    ], dim=0)
    
    score = predictor(out, edge_label_index)
    loss = F.binary_cross_entropy_with_logits(score, edge_label)

    loss.backward()
    optimizer.step()
    return loss.item()
    

In [4]:
@torch.no_grad()
def test(model, data, predictor):
    model.eval()

    edge_label_index = torch.cat([
        data.pos_edge_label_index, data.neg_edge_label_index
    ], dim=1)
    edge_label = torch.cat([
        data.pos_edge_label, data.neg_edge_label
    ], dim=0)

    out = model(data.x, data.edge_index)
    score = predictor(out, edge_label_index).cpu().numpy()
    auc = roc_auc_score(edge_label.cpu().numpy(), score)
    ap = average_precision_score(edge_label.cpu().numpy(), score)
    return auc, ap

In [5]:
train_data = torch.load('./data/Cora/split/train_data.pt').to(device)
val_data = torch.load('./data/Cora/split/val_data.pt').to(device)
test_data = torch.load('./data/Cora/split/test_data.pt').to(device)

model = ScoreGNN(config.data_init_num_features, hidden_dim = config.scoregnn.hidden_dim, 
                 output_dim = config.scoregnn.output_dim , num_layers = config.scoregnn.num_layers, 
                 dropout = config.scoregnn.dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.scoregnn.lr)

final_model = final_predictor = None
best_val_auc = final_test_auc = final_test_ap = 0

model.reset_parameters()

for epoch in range(1, 1 + config.scoregnn.epochs):
    loss = train(model, train_data, optimizer, config.predictor)
    train_auc, train_ap = test(model, train_data, config.predictor)
    val_auc, val_ap = test(model, val_data, config.predictor)
    test_auc, test_ap = test(model, test_data, config.predictor)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
        final_test_ap = test_ap
        final_model = copy.deepcopy(model)
        final_predictor = copy.deepcopy(config.predictor)

    logging.info(f'Epoch: {epoch:03d}, Loss: {loss:.4f} '
             f'Train_AUC: {train_auc:.4f}, Train_AP: {train_ap:.4f} '
             f'Val_AUC: {val_auc:.4f}, Val_AP: {val_ap:.4f} '
             f'Test_AUC: {test_auc:.4f}, Test_AP: {test_ap:.4f}')
    
logging.info(f'Final Test AUC: {final_test_auc:.4f}, AP: {final_test_ap:.4f}')

Epoch: 001, Loss: 0.6738 Train_AUC: 0.7983, Train_AP: 0.7755 Val_AUC: 0.7122, Val_AP: 0.7391 Test_AUC: 0.6824, Test_AP: 0.7110


Epoch: 002, Loss: 0.5769 Train_AUC: 0.7854, Train_AP: 0.7697 Val_AUC: 0.6947, Val_AP: 0.7279 Test_AUC: 0.6814, Test_AP: 0.7117


Epoch: 003, Loss: 0.4981 Train_AUC: 0.7768, Train_AP: 0.7649 Val_AUC: 0.6912, Val_AP: 0.7255 Test_AUC: 0.6814, Test_AP: 0.7112


Epoch: 004, Loss: 0.4335 Train_AUC: 0.7830, Train_AP: 0.7687 Val_AUC: 0.6977, Val_AP: 0.7292 Test_AUC: 0.6843, Test_AP: 0.7132


Epoch: 005, Loss: 0.4065 Train_AUC: 0.7869, Train_AP: 0.7704 Val_AUC: 0.6993, Val_AP: 0.7300 Test_AUC: 0.6843, Test_AP: 0.7129


...

Epoch: 197, Loss: 0.1910 Train_AUC: 1.0000, Train_AP: 1.0000 Val_AUC: 0.8401, Val_AP: 0.8588 Test_AUC: 0.8470, Test_AP: 0.8583


Epoch: 198, Loss: 0.1926 Train_AUC: 1.0000, Train_AP: 1.0000 Val_AUC: 0.8390, Val_AP: 0.8580 Test_AUC: 0.8462, Test_AP: 0.8558


Epoch: 199, Loss: 0.1911 Train_AUC: 1.0000, Train_AP: 1.0000 Val_AUC: 0.8382, Val_AP: 0.8569 Test_AUC: 0.8466, Test_AP: 0.8553


Epoch: 200, Loss: 0.1912 Train_AUC: 1.0000, Train_AP: 1.0000 Val_AUC: 0.8388, Val_AP: 0.8562 Test_AUC: 0.8475, Test_AP: 0.8571


Final Test AUC: 0.8682, AP: 0.8744


In [6]:
torch.save({
    'model': final_model.state_dict(),
    'predictor': final_predictor.state_dict()
}, './model/scoregnn.pth')